In [ ]:
import pandas as pd

In [ ]:
import seaborn as sns

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
from sklearn.preprocessing import StandardScaler

# Step 1: Load the dataset

In [ ]:
df = pd.read_csv('data.csv')

In [ ]:
print("First 5 rows of the dataset:")

In [ ]:
print(df.head())

In [ ]:
print("\nInformation about the dataset:")

In [ ]:
print(df.info())